In [2]:
import pandas as pd
import numpy as np
import os
import zipfile
from sklearn.model_selection import train_test_split

In [3]:
path = "C:\\Users\\Rohit Mhatre\\Downloads\\LendingClubs\\accepted_2007_to_2018q4.csv"
files = os.listdir(path)
print(files)

['accepted_2007_to_2018Q4.csv']


In [5]:
csv_path = "C:\\Users\\Rohit Mhatre\\Downloads\\LendingClubs\\accepted_2007_to_2018q4.csv\\accepted_2007_to_2018Q4.csv"

chunk_size = 100000
chunks = []
for chunk in pd.read_csv(csv_path,low_memory=False,chunksize = chunk_size):
    chunks.append(chunk)

ParserError: Error tokenizing data. C error: out of memory

In [5]:
data = pd.concat(chunks,ignore_index=True)

print(data.shape)
data.head()

(2260701, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
important_features = [
    'loan_amnt',         # Loan amount applied for
    'int_rate',          # Interest rate of the loan
    'installment',       # Monthly repayment amount
    'grade',             # Credit grade (A to G)
    'emp_length',        # Years of employment 
    'annual_inc',        # Borrower's annual income
    'dti',               # Debt-to-Income ratio (high = risk)
    'fico_range_low',    # Lower bound of FICO credit score
    'revol_util',        # Credit utilization rate (indicator of credit behavior)
    'total_rec_prncp',   # Total principal paid (loan repayment indicator)
    'purpose',           # Loan purpose (categorical - insights on default risk)
    'loan_status',       # Target variable (Fully Paid, Charged Off, Default)
    'verification_status', # Whether borrower's income is verified
    'last_pymnt_amnt',    # Last payment amount 
    'total_rec_int'       # Total interest received 
]
data = data[important_features]

In [7]:
data.head()

,loan_amnt,int_rate,installment,grade,emp_length,annual_inc,dti,fico_range_low,revol_util,total_rec_prncp,purpose,loan_status,verification_status,last_pymnt_amnt,total_rec_int
0,3600.0,13.99,123.03,C,10+ years,55000.0,5.91,675.0,29.7,3600.00,debt_consolidation,Fully Paid,Not Verified,122.67,821.72
1,24700.0,11.99,820.28,C,10+ years,65000.0,16.06,715.0,19.2,24700.00,small_business,Fully Paid,Not Verified,926.35,979.66
2,20000.0,10.78,432.66,B,10+ years,63000.0,10.78,695.0,56.2,20000.00,home_improvement,Fully Paid,Not Verified,15813.30,2705.92
3,35000.0,14.85,829.90,C,10+ years,110000.0,17.06,785.0,11.6,19102.35,debt_consolidation,Current,Source Verified,829.90,12361.66
4,10400.0,22.45,289.91,F,3 years,104433.0,25.37,695.0,64.5,10400.00,major_purchase,Fully Paid,Source Verified,10128.96,1340.50


In [8]:
data['loan_status'].unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [9]:
data = data[data['loan_status'].isin(['Fully Paid','Charged Off','Default'])]

data.dropna(axis=0,subset=['loan_status', 'annual_inc', 'int_rate', 'fico_range_low'],inplace=True)

data_sampled,_ = train_test_split(data,train_size = 90000,stratify=data['loan_status'],random_state=42)

print(data_sampled.shape)
data_sampled.head(2)

(90000, 15)


,loan_amnt,int_rate,installment,grade,emp_length,annual_inc,dti,fico_range_low,revol_util,total_rec_prncp,purpose,loan_status,verification_status,last_pymnt_amnt,total_rec_int
555456,5000.0,16.02,175.84,C,10+ years,65000.0,22.56,660.0,42.0,5000.00,debt_consolidation,Fully Paid,Not Verified,2776.11,498.09
1218856,8000.0,10.99,261.88,B,10+ years,60000.0,31.02,675.0,42.1,571.04,credit_card,Charged Off,Source Verified,261.88,214.60


In [10]:
data_sampled.to_csv("C:\\Users\\Rohit Mhatre\\Downloads\\LendinClub_reduced.csv",index=False)